Recarga de Modulos en cada ejecución

In [104]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Credenciales de conexión

In [142]:
'''
Lectura de Credenciales en config.json
'''
import json

with open('config.json', 'r') as configfile:
    config = json.load(configfile)
#print(config["sql_server"]["servidor"])


Importación de Módulos
Datos de Entrada

In [143]:
import Modulo.DBConnection as db
from Modulo.StructureMigration import Migration

#Datos de Entrada
origin_database = 'PRUEBA'
origin_schema ='dbo'
origin_table = 'prueba_datos_Sqlserver'

destiny_database = ''
destiny_schema = ''
destiny_table = origin_table

Creación de conexión a Bases de Datos (Origen y Destino)

In [144]:

#Conexión a Base de datos fuente (Origen de Datos)
origin_credentials = config["sqlserver"]
origin_connection = db.SQLServer(origin_credentials["user"], origin_credentials["password"], origin_credentials["server"], 'master', origin_credentials["port"])
origin_result = origin_connection.crear_conexion()

#Conexión a Base de datos destino
destiny_credentials = config["oracle"]
destiny_connection = db.Oracle(destiny_credentials["user"], destiny_credentials["password"], destiny_credentials["server"], destiny_credentials["servicename"], destiny_credentials["port"])
destiny_result = destiny_connection.crear_conexion()


In [133]:
query_oracle_sqlserver = '''
CREATE TABLE prueba_datos_Oracle (
    id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    numero_col NUMBER,
    varchar_col VARCHAR2(255),
    char_col CHAR(100),
    date_col DATE,
    timestamp_col TIMESTAMP,
    clob_col CLOB,
    blob_col BLOB,
    binary_float_col BINARY_FLOAT,
    binary_double_col BINARY_DOUBLE,
    interval_year_to_month_col INTERVAL YEAR TO MONTH,
    interval_day_to_second_col INTERVAL DAY TO SECOND,
    raw_col RAW(100),
    rowid_col ROWID
);
'''

query_sqlserver_oracle = '''
CREATE TABLE PRUEBA.dbo.prueba_datos_Sqlserver (
    id INT IDENTITY(1,1) PRIMARY KEY,
    numero_col INT,
    varchar_col VARCHAR(255),
    char_col CHAR(100),
    datetime_col DATETIME,
    datetime2_col DATETIME2,
    text_col TEXT,
    varbinary_col VARBINARY(MAX),
    float_col FLOAT,
    real_col REAL,
    date_col DATE,
    time_col TIME,
    uniqueidentifier_col UNIQUEIDENTIFIER,
    bit_col BIT
);
'''
#query = 'drop table prueba_datos_Oracle'
#result = origin_connection.ejecutar_sentencia(query_sqlserver_oracle)
#result = origin_connection.ejecutar_consulta('SELECT * FROM prueba_datos_Sqlserver')
#print(result)

True


Creación de Objeto de Migración

In [145]:

if origin_result == True and destiny_result ==True:
    print('Conexión Realizada con exito a Origen y Destino!')

    #Objeto de Migracion
    migracion = Migration()
    migracion.set_origin(origin_connection,origin_database, origin_schema, origin_table)
    migracion.set_destiny(destiny_connection,destiny_database, destiny_schema, destiny_table)
   
    print(f'La migración es la siguiente: {migracion.migration_type()}')

else:
    print('Error en la conexión:')
    if origin_result == True:
        print('\n', f'Conexión exitosa con el origen: {origin_connection.type}')
    else:
        print('\n', f'Conexión fallida con el origen: {origin_connection.type}')
        print(origin_result)

    if destiny_result == True:
        print('\n', f'Conexión exitosa con el destino: {destiny_connection.type}')
    else:
        print('\n', f'Conexión fallida con el destino: {destiny_connection.type}')
        print(destiny_result)

# structure = migracion.getorigin_Structure()
# structure.tail(5)
# query = migracion.gendestiny_Query()
# print(query)

Conexión Realizada con exito a Origen y Destino!
La migración es la siguiente: sqlserver --> oracle


In [149]:
#structure = migracion.get_origin_structure()
#structure
query = migracion.generate_destiny_create_table()
print(query)


CREATE TABLE prueba_datos_Sqlserver (
	id NUMBER(10) NOT NULL,
        numero_col NUMBER(10) NULL,
        varchar_col VARCHAR2(255) NULL,
        char_col CHAR(100) NULL,
        datetime_col DATE NULL,
        datetime2_col TIMESTAMP NULL,
        text_col CLOB NULL,
        varbinary_col BLOB NULL,
        float_col FLOAT NULL,
        real_col FLOAT NULL,
        date_col DATE NULL,
        time_col TIMESTAMP NULL,
        uniqueidentifier_col RAW(16) NULL,
        bit_col NUMBER(1) NULL
)
		


Ejecución de Migración de Estructura

In [139]:
ejecucion_migracion = migracion.run_structure_migration()

if ejecucion_migracion == True:
    print('Estructura creada con exito!')
else:
    print(f'Se encontro un error en la ejecución:\n{ejecucion_migracion}')

Estructura creada con exito!


In [ ]:
# SELECT 
#     c.column_name,
#     c.data_type,
#     c.character_maximum_length,
#     c.is_nullable,
#     CASE 
#         WHEN tc.constraint_type = 'PRIMARY KEY' THEN 'YES'
#         ELSE 'NO'
#     END AS IsPrimaryKey
# FROM 
#     information_schema.columns c
# LEFT JOIN 
#     information_schema.table_constraints tc
#     ON c.table_name = tc.table_name AND tc.constraint_type = 'PRIMARY KEY'
# WHERE 
#     c.table_name = 'DimEmployee';
